## Searching for bouts for a day of alsa recording

In [1]:
import os
import glob
import socket
import logging
import pickle
import numpy as np
import pandas as pd
from scipy.io import wavfile
from scipy import signal

### Fuck matplotlib, I'm using poltly now
from plotly.subplots import make_subplots
import plotly.graph_objects as go

from importlib import reload

logger = logging.getLogger()
handler = logging.StreamHandler()
formatter = logging.Formatter(
        '%(asctime)s %(name)-12s %(levelname)-8s %(message)s')
handler.setFormatter(formatter)
logger.addHandler(handler)
logger.setLevel(logging.INFO)

logger.info('Running on {}'.format(socket.gethostname()))

2021-07-23 12:04:57,473 root         INFO     Running on pakhi


In [2]:
from ceciestunepipe.file import bcistructure as et

### Get the file locations for a session (day) of recordings

In [46]:
reload(et)
sess_par = {'bird': 'z_r12r13_21',
           'sess': '2021-07-23'}


exp_struct = et.get_exp_struct(sess_par['bird'], sess_par['sess'], ephys_software='alsa')

raw_folder = exp_struct['folders']['alsa']
bouts_folder = os.path.join(exp_struct['folders']['derived'], 'bouts_ceciestunepipe')

In [47]:
exp_struct['folders']

{'bird': '/mnt/sphere/speech_bci/raw_data/s_b1253_21',
 'raw': '/mnt/sphere/speech_bci/raw_data/s_b1253_21/2021-07-23',
 'alsa': '/mnt/sphere/speech_bci/raw_data/s_b1253_21/2021-07-23/alsa',
 'kwik': '/scratch/earneodo/s_b1253_21/alsa/kwik/2021-07-23',
 'processed': '/mnt/sphere/speech_bci/processed_data/s_b1253_21/2021-07-23/alsa',
 'derived': '/mnt/sphere/speech_bci/derived_data/s_b1253_21/2021-07-23/alsa',
 'tmp': '/scratch/earneodo/tmp',
 'msort': '/scratch/earneodo/s_b1253_21/alsa/msort/2021-07-23',
 'ksort': '/scratch/earneodo/s_b1253_21/alsa/ksort/2021-07-23'}

### read a session's bouts pandas
- get the location of the file
- load the pandas pickle
- load the search/spectrogram parameters

In [48]:
from ceciestunepipe.util.sound import boutsearch as bs

In [49]:
### load the search/spectrogram parameters
### these functions are necessary to load the pickle files, which is bad.
### need to not write modules/functions in the pickle

# function for getting one channel out of a wave file
def read_wav_chan(wav_path: str, chan_id: int=0) -> tuple:
    s_f, x = wavfile.read(wav_path, mmap=True)
    return s_f, x[:, chan_id]

def sess_file_id(f_path):
    n = int(os.path.split(f_path)[1].split('-')[-1].split('.wav')[0])
    return n

hparams_file_path = os.path.join(bouts_folder, 'bout_search_params.pickle')
with open(hparams_file_path, 'rb') as fh:
    hparams = pickle.load(fh)

In [50]:
#bpd, x, p = bs.get_bouts_in_file(one_wav_path, hparams)

bouts_auto_file_path = os.path.join(bouts_folder, hparams['bout_auto_file'])

#load. It is important to reset index because the manual curation requires unique indexing
bpd = pd.read_pickle(bouts_auto_file_path).reset_index(drop=True)
bpd.head(1)

,start_ms,end_ms,start_sample,end_sample,p_step,rms_p,peak_p,bout_check,file,len_ms,syl_in,n_syl,peaks_p,n_peaks,l_p_ratio,waveform
0,1350515,1356095,64824720,65092560,"[42.625896104768216, 38.099377233589, 22.97955...",1.717928,77.854806,True,/mnt/sphere/speech_bci/raw_data/s_b1253_21/202...,5580,"[[1350515, 1350560], [1350695, 1350730], [1350...",12,"[5, 19, 36, 97, 110, 126, 178, 215, 395, 433, ...",22,253.636364,"[7, 7, 8, 7, 7, 7, 7, 8, 8, 9, 10, 8, 8, 8, 7,..."


###### if it wasnt saved (which is a bad mistake), read the sampling rate from the first file in the session

In [51]:
def sample_rate_from_wav(wav_path):
    x, sample_rate = wavfile.read(wav_path)
    return sample_rate

if hparams['sample_rate'] is None:
    one_wav_path = bpd.loc[0, 'file']
    logger.info('Sample rate not saved in parameters dict, searching it in ' + one_wav_path)
    hparams['sample_rate'] = sample_rate_from_wav(one_wav_path)

In [52]:
def cleanup(bout_pd: pd.DataFrame):
    ## check for empty waveforms (how woudld THAT happen???)
    bout_pd['valid_waveform'] = bout_pd['waveform'].apply(lambda x: (False if x.size==0 else True))
    
    # valid is & of all the validated criteria
    bout_pd['valid'] = bout_pd['valid_waveform']
    
    # drop not valid and reset index
    bout_pd.drop(bout_pd[bout_pd['valid']==False].index, inplace=True)
    bout_pd.reset_index(drop=True, inplace=True)

cleanup(bpd)

#### compute the spectrograms

In [53]:
bpd['spectrogram'] = bpd['waveform'].apply(lambda x: bs.gimmepower(x, hparams)[2])
logger.info('saving bout pandas with spectrogram to ' + bouts_auto_file_path)
bpd.to_pickle(bouts_auto_file_path)

2021-07-23 10:01:11,088 root         INFO     saving bout pandas with spectrogram to /mnt/sphere/speech_bci/derived_data/s_b1253_21/2021-07-23/alsa/bouts_ceciestunepipe/bout_auto.pickle


In [54]:
bpd

,start_ms,end_ms,start_sample,end_sample,p_step,rms_p,peak_p,bout_check,file,len_ms,syl_in,n_syl,peaks_p,n_peaks,l_p_ratio,waveform,valid_waveform,valid,spectrogram
0,1350515,1356095,64824720,65092560,"[42.625896104768216, 38.099377233589, 22.97955...",1.717928,77.854806,True,/mnt/sphere/speech_bci/raw_data/s_b1253_21/202...,5580,"[[1350515, 1350560], [1350695, 1350730], [1350...",12,"[5, 19, 36, 97, 110, 126, 178, 215, 395, 433, ...",22,253.636364,"[7, 7, 8, 7, 7, 7, 7, 8, 8, 9, 10, 8, 8, 8, 7,...",True,True,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
1,799105,808265,38357040,38796720,"[8.019162855890755, 22.015815801673362, 14.975...",1.910934,56.807087,True,/mnt/sphere/speech_bci/raw_data/s_b1253_21/202...,9160,"[[799105, 799140], [800190, 800245], [800490, ...",12,"[1, 45, 183, 218, 227, 278, 287, 296, 527, 537...",35,261.714286,"[2, 1, 1, 2, 2, 3, 3, 2, 2, 2, 3, 2, 2, 2, 2, ...",True,True,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
2,1376400,1381670,66067200,66320160,"[5.848350566500915, 2.9361233903608497, 7.5783...",1.910934,50.540186,True,/mnt/sphere/speech_bci/raw_data/s_b1253_21/202...,5270,"[[1376400, 1376445], [1377715, 1377755], [1377...",9,"[3, 59, 109, 245, 264, 306, 321, 328, 444, 454...",21,250.952381,"[6, 5, 5, 5, 4, 5, 5, 5, 5, 4, 5, 5, 5, 5, 4, ...",True,True,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
3,1511740,1517305,72563520,72830640,"[2.096464370577018, 8.58713345269559, 18.61735...",1.910934,86.582178,True,/mnt/sphere/speech_bci/raw_data/s_b1253_21/202...,5565,"[[1511740, 1511800], [1512895, 1512925], [1514...",8,"[3, 30, 43, 50, 83, 235, 376, 401, 452, 517, 5...",24,231.875000,"[5, 6, 5, 5, 4, 5, 6, 5, 6, 6, 5, 5, 5, 5, 4, ...",True,True,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
4,1569875,1575660,75354000,75631680,"[38.91167897619742, 25.656626259185348, 19.452...",1.588887,69.107965,True,/mnt/sphere/speech_bci/raw_data/s_b1253_21/202...,5785,"[[1569875, 1570030], [1570125, 1570175], [1570...",10,"[16, 27, 39, 51, 63, 75, 85, 92, 102, 115, 129...",25,231.400000,"[-3, -4, -4, -4, -5, -3, -3, -4, -4, -4, -5, -...",True,True,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
5,1343560,1386540,64490880,66553920,"[1.3815029128614749, 1.4730058512402793, 1.820...",3.565241,122.847502,True,/mnt/sphere/speech_bci/raw_data/s_b1253_21/202...,42980,"[[1343560, 1343635], [1343665, 1343710], [1345...",169,"[7, 27, 381, 391, 401, 722, 731, 745, 898, 912...",533,80.637899,"[3, 2, 3, 2, 3, 3, 3, 2, 3, 3, 3, 2, 2, 3, 2, ...",True,True,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
6,1544110,1588015,74117280,76224720,"[2.7818997755595984, 14.220558952586687, 20.09...",3.565241,133.995195,True,/mnt/sphere/speech_bci/raw_data/s_b1253_21/202...,43905,"[[1544110, 1544190], [1544330, 1544370], [1544...",183,"[7, 47, 57, 68, 88, 287, 318, 337, 350, 360, 3...",571,76.891419,"[4, 4, 3, 2, 1, 1, 0, -1, -3, -2, -2, -4, -4, ...",True,True,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."


## inspect the bouts and curate them

#### visualize one bout

In [55]:
bpd.iloc[0]

start_ms                                                    1350515
end_ms                                                      1356095
start_sample                                               64824720
end_sample                                                 65092560
p_step            [42.625896104768216, 38.099377233589, 22.97955...
rms_p                                                      1.717928
peak_p                                                    77.854806
bout_check                                                     True
file              /mnt/sphere/speech_bci/raw_data/s_b1253_21/202...
len_ms                                                         5580
syl_in            [[1350515, 1350560], [1350695, 1350730], [1350...
n_syl                                                            12
peaks_p           [5, 19, 36, 97, 110, 126, 178, 215, 395, 433, ...
n_peaks                                                          22
l_p_ratio                                       

In [56]:
import plotly.express as px
import plotly.graph_objects as go

from ipywidgets import widgets

In [57]:
def viz_one_bout(df: pd.Series, sub_sample=10):
    # get the power and the spectrogram
    sxx = df['spectrogram'][:, ::sub_sample]
    x = df['waveform'][::sub_sample]
    
    # the trace
    tr_waveform = go.Scatter(y=x)
    figwidg_waveform = go.FigureWidget(data=[tr_waveform],
                                      layout= {'height': 300,'width':1000})

    # the spectrogram
    fig_spectrogram = px.imshow(sxx, 
                                     labels={}, 
                                     color_continuous_scale='Inferno',
                                    aspect='auto')

    fig_spectrogram.update_layout(width=1000, height=300, coloraxis_showscale=False)
    fig_spectrogram.update_xaxes(showticklabels=False)
    fig_spectrogram.update_yaxes(showticklabels=False)
    figwidg_spectrogram = go.FigureWidget(fig_spectrogram)
    
    display(widgets.VBox([figwidg_waveform,
              figwidg_spectrogram]))
    

viz_one_bout(bpd.iloc[0])

    'data': [{'type': 'scatter',
              'uid': '7973ef68-9a97-4bf3-84b9-2…

In [58]:
bpd.head(2)

,start_ms,end_ms,start_sample,end_sample,p_step,rms_p,peak_p,bout_check,file,len_ms,syl_in,n_syl,peaks_p,n_peaks,l_p_ratio,waveform,valid_waveform,valid,spectrogram
0,1350515,1356095,64824720,65092560,"[42.625896104768216, 38.099377233589, 22.97955...",1.717928,77.854806,True,/mnt/sphere/speech_bci/raw_data/s_b1253_21/202...,5580,"[[1350515, 1350560], [1350695, 1350730], [1350...",12,"[5, 19, 36, 97, 110, 126, 178, 215, 395, 433, ...",22,253.636364,"[7, 7, 8, 7, 7, 7, 7, 8, 8, 9, 10, 8, 8, 8, 7,...",True,True,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
1,799105,808265,38357040,38796720,"[8.019162855890755, 22.015815801673362, 14.975...",1.910934,56.807087,True,/mnt/sphere/speech_bci/raw_data/s_b1253_21/202...,9160,"[[799105, 799140], [800190, 800245], [800490, ...",12,"[1, 45, 183, 218, 227, 278, 287, 296, 527, 537...",35,261.714286,"[2, 1, 1, 2, 2, 3, 3, 2, 2, 2, 3, 2, 2, 2, 2, ...",True,True,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."


## use it in a widget


#### add a 'confusing' label, for not/sure/mixed.
we want to avoid having things we are not sure of in the training dataset

In [59]:
bpd.reset_index(drop=True, inplace=True)

In [60]:
## Set confusing by default, will only be False once asserted bout/or not
bpd['confusing'] = True
bpd['bout_check'] = False

In [61]:
### Create a counter object (count goes 1:1 to DataFrame index)
from traitlets import CInt, link

class Counter(widgets.DOMWidget):
    value = CInt(0)
    value.tag(sync=True)

In [62]:
class VizBout():
    def __init__(self, hparams, bouts_pd):
        self.bout = None
        self.bouts_pd = bouts_pd
        self.bout_series = None
        self.is_bout = None
        self.is_confusing = None
        
        self.bout_counter = None
        self.bout_id = None
        
        self.buttons = {}
        self.m_pick = None
        
        
        self.fig_waveform = None
        self.fig_spectrogram = None
        
        self.figwidg_waveform = None
        self.figwidg_spectrogram = None
        
        self.fig_width = 2
        self.sub_sample = 10
        
        self.x = None
        self.sxx = None
        self.tr_waveform = None
        
        self.s_f = hparams['sample_rate']
        
        self.init_fig()
        self.init_widget()
        self.show()
        
    def init_fig(self):
        # the trace
        self.tr_waveform = go.Scatter(y=np.zeros(500))
        self.figwidg_waveform = go.FigureWidget(data=[self.tr_waveform],
                                               layout={'width': 1000, 'height':300})
        
        # the spectrogram
        self.fig_spectrogram = px.imshow(np.random.rand(500, 500), 
                                         labels={}, 
                                         color_continuous_scale='Inferno',
                                        aspect='auto')
        
        self.fig_spectrogram.update_layout(width=1000, height=300, coloraxis_showscale=False)
        self.fig_spectrogram.update_xaxes(showticklabels=False)
        self.fig_spectrogram.update_yaxes(showticklabels=False)
        self.figwidg_spectrogram = go.FigureWidget(self.fig_spectrogram)
        
        
    def init_widget(self):
        # declare elements
        # lay them out
        #
        
        self.bout_counter = Counter()
        self.is_bout = widgets.Checkbox(description='is bout')
        self.is_confusing = widgets.Checkbox(description='Not sure or mixed')
        
        self.buttons['Next'] = widgets.Button(description="Next", button_style='info',
                                              icon='plus')   
        self.buttons['Prev'] = widgets.Button(description="Prev", button_style='warning',
                                              icon='minus')
        self.buttons['Check'] = widgets.Button(description="Check", button_style='success', 
                                               icon='check')
        self.buttons['Uncheck'] = widgets.Button(description="Uncheck", button_style='danger',
                                                 icon='wrong')
        
        [b.on_click(self.button_click) for b in self.buttons.values()]
        
        left_box = widgets.VBox([self.buttons['Prev'], self.buttons['Uncheck']])
        right_box = widgets.VBox([self.buttons['Next'], self.buttons['Check']])
        button_box = widgets.HBox([left_box, right_box])

        self.m_pick = widgets.IntSlider(value=0, min=0, max=self.bouts_pd.index.size-1,step=1, 
                                        description="Bout candidate index")
        
        
        control_box = widgets.HBox([button_box,
                                  widgets.VBox([self.is_bout, self.is_confusing]),
                                    self.m_pick])
        
        link((self.m_pick, 'value'), (self.bout_counter, 'value'))

        self.update_bout()
        
        self.is_bout.observe(self.bout_checked, names='value')
        self.is_confusing.observe(self.confusing_checked, names='value')
        self.m_pick.observe(self.slider_change, names='value')
        
        all_containers = widgets.VBox([control_box, 
                                       self.figwidg_waveform, self.figwidg_spectrogram])
        display(all_containers)
#         display(button_box)
#         display(self.m_pick)
#         display(self.is_bout)
#         display(self.fig)
        
    def button_click(self, button):        
        self.bout_id = self.bout_counter.value
        curr_bout = self.bout_counter
        
        if button.description == 'Next':
            curr_bout.value += 1
        elif button.description == 'Prev':
            curr_bout.value -= 1
        elif button.description == 'Check':
            self.bouts_pd.loc[self.bout_id, 'bout_check'] = True
            self.bouts_pd.loc[self.bout_id, 'confusing'] = False
            curr_bout.value += 1
        elif button.description == 'Uncheck':
            self.bouts_pd.loc[self.bout_id, 'bout_check'] = False
            self.bouts_pd.loc[self.bout_id, 'confusing'] = False
            curr_bout.value += 1
        
        # handle the edges of the counter
        if curr_bout.value > self.m_pick.max:
            curr_bout.value = 0
            
        if curr_bout.value < self.m_pick.min:
            curr_bout.value = self.m_pick.max
    
    def slider_change(self, change):
        #logger.info('slider changed')
        #self.bout_counter = change.new
        #clear_output(True)
        self.update_bout()
        self.show()
            
    def bout_checked(self, bc):
#         print "bout checked"
#         print bc['new']
#         print self.motiff
        self.bouts_pd.loc[self.bout_id, 'bout_check'] = bc['new']
    
    def confusing_checked(self, bc):
#         print "bout checked"
#         print bc['new']
#         print self.motiff
        self.bouts_pd.loc[self.bout_id, 'confusing'] = bc['new']
    
    def update_bout(self):
        self.bout_id = self.bout_counter.value
        self.bout_series = self.bouts_pd.iloc[self.bout_id]
        
        self.is_bout.value = bool(self.bout_series['bout_check'])
        self.is_confusing.value = bool(self.bout_series['confusing'])
        
        self.x = self.bout_series['waveform'][::self.sub_sample]
        self.sxx = self.bout_series['spectrogram'][::self.sub_sample]
        
    def show(self):
        #self.fig.clf()
        #self.init_fig()
        # update
#         self.update_bout()
        #plot
        #logger.info('showing')
        
        # Show the figures
        with self.figwidg_waveform.batch_update():
            self.figwidg_waveform.data[0].y = self.x
            self.figwidg_waveform.data[0].x = np.arange(self.x.size) * self.sub_sample / self.s_f 
            
        with self.figwidg_spectrogram.batch_update():
            self.figwidg_spectrogram.data[0].z = np.sqrt(self.sxx[::-1])
            
        

viz_bout = VizBout(hparams, bpd)

In [43]:
np.where(viz_bout.bouts_pd['bout_check']==True)[0].size

67

### save it

In [44]:
hparams

{'num_freq': 1024,
 'preemphasis': 0.97,
 'frame_shift_ms': 5,
 'frame_length_ms': 10,
 'min_level_db': -55,
 'ref_level_db': 110,
 'sample_rate': 48000,
 'mel_filter': False,
 'num_mels': 1024,
 'fmin': 500,
 'fmax': 12000,
 'max_iters': 200,
 'griffin_lim_iters': 20,
 'power': 1.5,
 'read_wav_fun': <function __main__.read_wav_chan(wav_path: str, chan_id: int = 0) -> tuple>,
 'file_order_fun': <function __main__.sess_file_id(f_path)>,
 'min_segment': 30,
 'min_silence': 2000,
 'min_bout': 5000,
 'peak_thresh_rms': 0.55,
 'thresh_rms': 0.25,
 'mean_syl_rms_thresh': 0.3,
 'max_bout': 120000,
 'l_p_r_thresh': 100,
 'waveform_edges': 1000,
 'bout_auto_file': 'bout_auto.pickle',
 'bout_curated_file': 'bout_checked.pickle'}

In [45]:
### get the curated file path
##save to the curated file path
bouts_curated_file_path = os.path.join(bouts_folder, hparams['bout_curated_file'])

viz_bout.bouts_pd.to_pickle(bouts_curated_file_path)
logger.info('saved curated bout pandas to pickle {}'.format(bouts_curated_file_path))

2021-07-23 00:40:59,348 root         INFO     saved curated bout pandas to pickle /mnt/sphere/speech_bci/derived_data/s_b1253_21/2021-07-22/alsa/bouts_ceciestunepipe/bout_checked.pickle


In [23]:
viz_bout.bouts_pd.head(5)

,start_ms,end_ms,start_sample,end_sample,p_step,rms_p,peak_p,bout_check,file,len_ms,syl_in,n_syl,peaks_p,n_peaks,l_p_ratio,waveform,valid_waveform,valid,spectrogram,confusing
0,1617980,1663710,77663040,79858080,"[0.4687655876424791, 0.49630900845550874, 0.63...",1.707817,112.871502,True,/mnt/sphere/speech_bci/raw_data/s_b1253_21/202...,45730,"[[1617980, 1618115], [1618295, 1618370], [1619...",199,"[11, 25, 43, 70, 308, 331, 339, 388, 403, 597,...",607,75.337727,"[2, 2, 2, 3, 2, 2, 3, 2, 1, 2, 2, 1, 2, 2, 2, ...",True,True,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",False
1,1485950,1491495,71325600,71591760,"[11.595448087337415, 11.793765860154966, 19.14...",1.701865,106.035861,False,/mnt/sphere/speech_bci/raw_data/s_b1253_21/202...,5545,"[[1485950, 1485980], [1487705, 1487785], [1489...",12,"[2, 10, 21, 29, 45, 75, 82, 118, 131, 140, 171...",58,95.603448,"[-58, -41, -35, -36, -23, -29, -53, -53, -46, ...",True,True,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",False
2,1631405,1641190,78307440,78777120,"[51.42436629365317, 31.13508861353894, 13.4164...",1.701865,105.255788,False,/mnt/sphere/speech_bci/raw_data/s_b1253_21/202...,9785,"[[1631405, 1631480], [1632590, 1632655], [1633...",27,"[7, 14, 55, 73, 93, 101, 216, 244, 280, 335, 3...",94,104.095745,"[9, 10, 11, 10, 13, 12, 11, 11, 10, 12, 9, 8, ...",True,True,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",False
3,476945,483855,22893360,23225040,"[3.4742450176737103, 1.7577857659079832, 5.450...",1.495082,91.617017,False,/mnt/sphere/speech_bci/raw_data/s_b1253_21/202...,6910,"[[476945, 476995], [478780, 478815], [479225, ...",10,"[7, 30, 58, 96, 115, 194, 209, 216, 249, 340, ...",54,127.962963,"[-9, -5, -7, -9, -5, -9, -10, -8, -11, -9, -10...",True,True,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",False
4,122870,128745,5897760,6179760,"[32.32082327876065, 49.48996185549777, 26.6768...",2.282122,140.133193,False,/mnt/sphere/speech_bci/raw_data/s_b1253_21/202...,5875,"[[122870, 122900], [123380, 123465], [124275, ...",15,"[1, 15, 92, 105, 112, 138, 147, 154, 226, 240,...",62,94.758065,"[41, 31, 28, 30, -9, 2, 12, 16, 55, 73, 83, 35...",True,True,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",False
